In [ ]:
import os, sys 
IMATOOLS_DIR = os.getcwd()+'/..'
sys.path.insert(1, IMATOOLS_DIR)
from imatools.common import ioutils as iou

def mshz_cmd(png_file, msh_file, dat_file, mshz_state='') : 
    cmd  = 'meshalyzer '
    cmd += '--PNGfile=' + png_file + ' ' 
    cmd += msh_file + ' '
    cmd += dat_file + ' ' 
    cmd += mshz_state 

    return cmd


In [ ]:
import numpy as np 

# Locations of hard drive based on platform
dirdic = {'macOS': '/Volumes/sandisk',
          'Linux': '/media/jsl19/sandisk',
          'Windows': 'D:/'}

p2f = iou.fullfile(dirdic[iou.chooseplatform()], '01_atrialfibres/06_Reproducibility/05_UserProjects')

square_dir  = '007_simulations'
sims_dir = '008_simulation_results'
output_dir = '009_simulation_images'

N=np.linspace(1,100,num=100, dtype=int)

in_N = ['M' + iou.num2padstr(n) for n in N]
si_N = ['M' + str(n) for n in N]


In [ ]:
list_files = ['LAT_RSPV_1', 'LAT_RSPV_l', 'PSNode', 'PSNodeSmooth']

for ix in range(len(si_N)) :
    sdir = si_N[ix]
    sdir_2d = in_N[ix]

    print(sdir) 

    # mesh 
    msh = iou.fullfile(p2f, sims_dir, sdir, 'Monolayer') 
    msh_2d = iou.fullfile(p2f, square_dir, sdir_2d, 'Labelled_Coords_2D_Rescaling_v3_C')

    # dat 
    dat_list = [iou.fullfile(p2f, sims_dir, sdir, l+'.dat') for l in list_files] 

    # png 
    png_list = [iou.fullfile(p2f, output_dir , l ,'3D', sdir) for l in list_files]
    png_list_2d = [iou.fullfile(p2f, output_dir , l ,'2D', sdir) for l in list_files]

    for x in range(len(list_files)) : 
        mshz_state_file = iou.fullfile(p2f, list_files[x]+'.mshz')
        # print(mshz_cmd(png_list[x], msh, dat_list[x], mshz_state=mshz_state_file))
        os.system(mshz_cmd(png_list[x], msh, dat_list[x], mshz_state=mshz_state_file))
        # print(mshz_cmd(png_list_2d[x] + '_2D', msh_2d, dat_list[x], mshz_state=mshz_state_file))
        os.system(mshz_cmd(png_list_2d[x] + '_2D', msh_2d, dat_list[x], mshz_state=mshz_state_file))




In [ ]:
import numpy as np

list_files = ['LAT_RSPV_1', 'LAT_RSPV_l', 'PSNode', 'PSNodeSmooth']

lat_l = np.zeros((len(si_N), 2))
lat_1 = np.zeros((len(si_N), 2))
psn = np.zeros((len(si_N), 2))
psns = np.zeros((len(si_N), 2))

for ix in range(len(si_N)) : 
    # ix =0 
    print(si_N[ix])
    sdir = si_N[ix]
    sdir_2d = in_N[ix]
    dat_list = [iou.fullfile(p2f, sims_dir, sdir, l+'.dat') for l in list_files] 

    l = np.loadtxt(dat_list[0])
    lat_1[ix, :] = [np.min(l), np.max(l)]

    l = np.loadtxt(dat_list[1])
    lat_l[ix, :] = [np.min(l), np.max(l)]

    l = np.loadtxt(dat_list[2])
    psn[ix, :] = [np.min(l), np.max(l)]

    l = np.loadtxt(dat_list[3])
    psns[ix, :] = [np.min(l), np.max(l)]

iou.cout('LAT_RSPV_1 - min: {} max: {}'.format(np.min(lat_1), np.max(lat_1)))
iou.cout('LAT_RSPV_l - min: {} max: {}'.format(np.min(lat_l), np.max(lat_l)))
iou.cout('PSNode - min: {} max: {}'.format(np.min(psn), np.max(psn)))
iou.cout('PSNodeSmooth - min: {} max: {}'.format(np.min(psns), np.max(psns)))

In [ ]:
import pandas as pd 

df = pd.DataFrame(columns=['ID', 'original_path', 'simulation_path', 'user', 'patient', 'mode', 'processing']) 

for ix in range (len(in_N)) : 
    sdir_2d = in_N[ix]
    txt_file = iou.fullfile(p2f, square_dir, sdir_2d, 'README.txt')

    case_info = pd.read_csv(txt_file,index_col=0, header=None, skipinitialspace=True).T
    case_info

    df = pd.concat([df, case_info])

new_col = ['$AFIB_REPROD/008_simulation_results/' + n for n in si_N]
df.insert(loc=int(3), column='result_path', value=new_col)

out_file = iou.fullfile(p2f, 'simulations_paths.csv')
df.to_csv(out_file, index=False)